In [1]:
!nvidia-smi

Sun Jun  5 13:03:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 35.3 MB/s 
     |████████████████████████████████| 6.6 MB 58.3 MB/s 
     |████████████████████████████████| 86 kB 6.0 MB/s 
     |████████████████████████████████| 596 kB 60.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Create inputs for Transformers

In [1]:
import create_dataloader
from concat_four_last_layers import *
import pandas as pd
import torch
import os
from os.path import join
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoConfig,get_linear_schedule_with_warmup

In [3]:
pre_dataset_directory="/content/drive/MyDrive/en-vi-nli/pre_dataset"

In [4]:
train=pd.read_csv(f'{pre_dataset_directory}/train_100_60k.csv')
valid=pd.read_csv(f'{pre_dataset_directory}/valid.csv')

In [5]:
train['premise']=train['premise'].str.lower()

In [8]:
valid['premise']=valid['premise'].str.lower()

In [7]:
train['hypothesis']=train['hypothesis'].str.lower()

In [9]:
valid['hypothesis']=valid['hypothesis'].str.lower()

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
train

,premise,hypothesis,label,premise_language,hypothesis_language,premise_length,hypothesis_length,total_length
0,i know that is only part of problem,ma túy chỉ là một phần của vấn đề,1,en,vi,8,9,17
1,lanny davis closest thing to official source w...,lanny davis xin lỗi đã xin lỗi,2,en,vi,18,7,25
2,những gia đình lớn sống ở các khu vực cao chia...,gia chia sẻ a à,0,vi,vi,21,5,26
3,in 1927 merchant james oviatt was entranced wi...,kiến trúc của paris chưa bao giờ gây ấn tượng ...,2,en,vi,16,18,34
4,chúng tôi lưu cùng một container nhựa và chúng...,chúng tôi thường lưu và mang nó đến nhà của ch...,1,vi,vi,27,12,39
...,...,...,...,...,...,...,...,...
59995,find bus station sultan suleiman street betwee...,đường suleiman chưa bao giờ có một trạm xe buýt,2,en,vi,12,10,22
59996,the door on left beyond hearth leads to valide...,phòng ngủ của sultan có giường gốc,1,en,vi,26,7,33
59997,he believed that data on prevalence and severi...,anh ta không có dữ liệu nào để trả lại lời kha...,2,en,vi,17,15,32
59998,đó là lý do tốt nhất daniel đã nòi,daniel nói điều đó là tốt nhất,0,vi,vi,9,7,16


In [11]:
valid

,premise,hypothesis,label,premise_language,hypothesis_language,premise_length,hypothesis_length,total_length
0,and he said mama i am home,ngay khi xuống xe buýt của trường anh ấy gọi c...,1,en,vi,7,12,19
1,and he said mama i am home,anh không nói một lời nào,2,en,vi,7,6,13
2,and he said mama i am home,anh nói với mẹ rằng anh đã về nhà,0,en,vi,7,9,16
3,i did not know what i was going for anything s...,tôi chưa bao giờ đến washington nên khi tôi đư...,1,en,vi,19,22,41
4,i did not know what i was going for anything s...,tôi biết chính xác những gì tôi cần làm khi tô...,2,en,vi,19,15,34
...,...,...,...,...,...,...,...,...
4975,từ feisty hăng hái cũng giống như fizzle cháy ...,fiesty đã tồn tại được 100 năm,1,vi,vi,29,7,36
4976,từ feisty hăng hái cũng giống như fizzle cháy ...,sự giận dữ không liên quan đến việc fisten,2,vi,vi,29,9,38
4977,dù lời tuyên bố sẽ tốt hơn câu trả lời gợi lên...,tuyên bố cung cấp thêm chi tiết,1,vi,vi,18,7,25
4978,dù lời tuyên bố sẽ tốt hơn câu trả lời gợi lên...,khẳng định không tốt hơn,2,vi,vi,18,5,23


In [13]:
train_batch_size=16
train_premises=train['premise'].tolist()
train_hypothesises=train['hypothesis'].tolist()
train_labels=train['label'].tolist()

In [14]:
valid_batch_size=16
valid_premises=valid['premise'].tolist()
valid_hypothesises=valid['hypothesis'].tolist()
valid_labels=valid['label'].tolist()

In [15]:
xlm_name='xlm-roberta-large'
xlm_tokenizer=AutoTokenizer.from_pretrained(xlm_name)

In [ ]:
rembert_name='google/rembert'
rembert_tokenizer=AutoTokenizer.from_pretrained(rembert_name)

In [16]:
max_length=100

In [17]:
xlm_train_dataloader=create_dataloader.create_dataloader(xlm_tokenizer,train_premises,train_hypothesises,train_labels,'roberta',max_length,train_batch_size,device=device,shuffle=False)

  0%|          | 0/60000 [00:00<?, ?it/s]

In [18]:
xlm_valid_dataloader=create_dataloader.create_dataloader(xlm_tokenizer,valid_premises,valid_hypothesises,valid_labels,'roberta',max_length,train_batch_size,device=device,shuffle=False)

  0%|          | 0/4980 [00:00<?, ?it/s]

In [ ]:
rembert_train_dataloader=create_dataloader.create_dataloader(rembert_tokenizer,train_premises,train_hypothesises,train_labels,'roberta',max_length,train_batch_size,device=device,shuffle=False)

  0%|          | 0/100000 [00:00<?, ?it/s]

In [ ]:
rembert_valid_dataloader=create_dataloader.create_dataloader(rembert_tokenizer,valid_premises,valid_hypothesises,valid_labels,'roberta',max_length,train_batch_size,device=device,shuffle=False)

  0%|          | 0/4980 [00:00<?, ?it/s]

In [19]:
labels=[0,1,2]

In [20]:
epochs=30
init_lr=1e-5
eps =1e-8

In [21]:
num_labels=len(labels)
num_labels

3

In [22]:
classifier_dropout=0.1

In [ ]:
rembert_config=AutoConfig.from_pretrained(rembert_name)
rembert_config.classifier_dropout=classifier_dropout
rembert_config.num_labels=num_labels
rembert_config

In [24]:
xlm_config=AutoConfig.from_pretrained(xlm_name)
xlm_config.classifier_dropout=classifier_dropout
xlm_config.num_labels=num_labels
xlm_config

XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-large",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": 0.1,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.19.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

In [25]:
model_directory='/content/drive/MyDrive/en-vi-nli/ensemble'
os.makedirs(model_directory,exist_ok=True)

In [ ]:
rembert_config.save_pretrained(join(model_directory,'rember'))

In [26]:
xlm_config.save_pretrained(join(model_directory,'xlm-roberta-large'))

In [27]:
xlm_model=XLMRoBERTa4LastLayersForClassification.from_pretrained(xlm_name,config=xlm_config).to(device)

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRoBERTa4LastLayersForClassification: ['lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRoBERTa4LastLayersForClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRoBERTa4LastLayersForClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRoBERTa4LastLayersForClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['cl

In [ ]:
rembert=RemBert4LastLayersForClassification.from_pretrained(rembert_name,config=rembert_config).to(device)

In [28]:
import gc
gc.collect()

1135

In [29]:
xlm_optimizer=torch.optim.AdamW(xlm_model.parameters(),lr=init_lr,weight_decay =0.01,eps=eps)
# rembert_optimizer=torch.optim.AdamW(rembert.parameters(),lr=init_lr,weight_decay =0.01,eps=eps)

In [30]:
def save_model(epoch,model,optimizer,scheduler,training_loss,valid_loss,train_acc,valid_acc,path,step=None):
  torch.save({
          'epoch': epoch,
          'model_state_dict': model,
          'optimizer_state_dict': optimizer,
          'scheduler_state_dict': scheduler,
          'training_loss': training_loss,
          'valid_loss': valid_loss,
          'train_acc': train_acc,
          'valid_acc': valid_acc,
          'step':step
          }, join(path,f'ckpt{epoch}.pt'))

In [31]:
# len(xlm_train_dataloader),len(rembert_train_dataloader)

In [32]:
loss_fn=nn.CrossEntropyLoss()

In [33]:
from sklearn.metrics import *


In [34]:
def run_train(model,optimizer,dataloader,lr_scheduler=None):
  model.train(True)

  loop = tqdm(dataloader)
  loss=0
  true=[]
  pred=[]
  for batch in dataloader:
    optimizer.zero_grad()
    inputs=batch['inputs']
    inputs.update({"labels":batch['labels']})
    labels=batch['labels']
    true+=labels.tolist()

    logits=model(**inputs)

    _loss=loss_fn(logits.view(-1, num_labels), labels.view(-1))
    _loss.backward()
    loss+=_loss.item()

    probs=torch.nn.functional.softmax(logits,dim=-1)
    _pred=torch.argmax(probs,dim=-1).tolist()
    pred+=_pred
    
    optimizer.step()
    if lr_scheduler!=None:
      lr_scheduler.step()
    loop.set_postfix({f'train_loss_per_batch':_loss.item()})
    loop.update()      
  acc=accuracy_score(true,pred)
  return loss/len(dataloader),acc

In [35]:
def run_valid(model,dataloader):
  model.eval()
  loss=0
  pred=[]
  true=[]
  with torch.no_grad():
    for batch in dataloader:
      inputs=batch['inputs']
      inputs.update({"labels":batch['labels']})
      labels=batch['labels']
      true+=labels.tolist()

      logits=model(**inputs)

      _loss=loss_fn(logits.view(-1, num_labels), labels.view(-1))
      loss+=_loss.item()
      
      probs=torch.nn.functional.softmax(logits,dim=-1)
      _pred=torch.argmax(probs,dim=-1).tolist()
      pred+=_pred
      

  acc=accuracy_score(true,pred)

  return loss/len(dataloader),acc

In [36]:
import gc
gc.collect()

102

In [37]:
lr_scheduler=None

In [38]:
model_directory='/content/drive/MyDrive/en-vi-nli/models/concat-4-last-layers-xlm-roberta-large'
os.makedirs(model_directory,exist_ok=True)
model_directory

'/content/drive/MyDrive/en-vi-nli/models/concat-4-last-layers-xlm-roberta-large'

In [ ]:
train_loss_per_step=[]
valid_loss_per_step=[]
train_acc_per_step=[]
valid_acc_per_step=[]
# min_loss=np.inf
best_acc=0
count_stopping=0
for epoch in range(epochs):

  train_loss,train_acc=run_train(xlm_model,xlm_optimizer,xlm_train_dataloader,lr_scheduler)
  
  valid_loss,valid_acc=run_valid(xlm_model,xlm_valid_dataloader)

  train_loss_per_step.append(train_loss)
  valid_loss_per_step.append(valid_loss)

  train_acc_per_step.append(train_acc)
  valid_acc_per_step.append(valid_acc)
  
  print(f'epoch: {epoch} train_loss: {train_loss} valid_loss: {valid_loss} train_acc: {train_acc} valid_acc: {valid_acc}')
  # if valid_loss<min_loss:
  if best_acc<valid_acc:
    # min_loss=valid_loss
    best_acc=valid_acc
    save_model(epoch,xlm_model.state_dict(),xlm_optimizer.state_dict(),lr_scheduler,train_loss,valid_loss,train_acc,valid_acc,model_directory)
    count_stopping=0
  else:
    count_stopping+=1
  
  if count_stopping>5:
    break


  0%|          | 0/3778 [00:00<?, ?it/s]